In [78]:
import torch 
import numpy as np
from PIL import Image
from glob import glob 
from tqdm import tqdm
from torch.utils.data import DataLoader


In [79]:
files = glob("../Dataset/train/*.jpg")

In [80]:
from torch.utils.data import Dataset
import torch 
from torchvision import transforms
from PIL import Image
import os


class SgranDataset(Dataset):

    def __init__(self, dir,hr_shape = (1024, 1024)):
        super(SgranDataset, self).__init__()

        self.hr_height, self.hr_width = hr_shape
        self.dir = dir

        self.lr_transforms = transforms.Compose([
            transforms.Resize((self.hr_height//4,  self.hr_width//4), Image.BICUBIC),
            transforms.ToTensor(),
            
        ])
        
        self.hr_transforms = transforms.Compose([
            transforms.Resize((self.hr_height,  self.hr_width), Image.BICUBIC),
            transforms.ToTensor(),
            
        ])


    def __len__(self):
        return len(files)


    def __getitem__(self, index):
        img = Image.open(files[index])
        img_lr = self.lr_transforms(img)
        img_hr = self.hr_transforms(img)
        return img_lr, img_hr



In [77]:

loader = DataLoader(SgranDataset(dir = "../Dataset/train"), batch_size=8)

nimages = 0
mean = 0.
std = 0.
for batch, _ in tqdm(loader):
    # Rearrange batch to be the shape of [B, C, W * H]
    batch = batch.view(batch.size(0), batch.size(1), -1)
    # Update total number of images
    nimages += batch.size(0)
    # Compute mean and std here
    mean += batch.mean(2).sum(0) 
    std += batch.std(2).sum(0)

# Final step
mean /= nimages
std /= nimages

print(mean)
print(std)

100%|██████████| 257/257 [00:41<00:00,  6.14it/s]

tensor([0.4262, 0.4133, 0.3996])
tensor([0.2174, 0.2022, 0.2092])
